<a href="https://colab.research.google.com/github/samhithabodangi/Organ-Rejection-Model/blob/main/Data%20Processing/fastaProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing fasta File from IPD/IMGT-HLA

In [ ]:
import ftplib
import os

## Accessing File From Public ftp Server

In [ ]:
ftp_server = 'ftp.cse.buffalo.edu'
ftp_session= ftplib.FTP(ftp_server)
ftp_session.login()

'230 Login successful.'

In [ ]:
ftp_host = 'ftp.ebi.ac.uk'
ftp_user = 'anonymous'
ftp_password = 'your@email.com'

ftp = ftplib.FTP(ftp_host)
ftp.login(user=ftp_user, passwd=ftp_password)

'230 Login successful.'

In [ ]:
ftp.cwd('/pub/databases/ipd/imgt/hla/')

'250 Directory successfully changed.'

In [ ]:
ftp.retrlines('LIST')

-rw-r--r--    1 ftp      ftp      10509655 Jan 16 15:56 Alignments_Rel_3550.zip
-rw-r--r--    1 ftp      ftp       1753288 Jan 16 15:56 Allele_status.txt
-rw-r--r--    1 ftp      ftp        789558 Jan 16 15:56 Allelelist.txt
-rw-r--r--    1 ftp      ftp      20839360 Jan 16 15:56 Allelelist_history.txt
-rw-r--r--    1 ftp      ftp         22284 Jan 16 15:56 Deleted_alleles.txt
-rw-r--r--    1 ftp      ftp          4870 Apr 17  2023 LICENCE.md
-rw-r--r--    1 ftp      ftp         29170 Apr 17  2023 Manual.md
-rw-r--r--    1 ftp      ftp        143518 Nov 30  2022 Nomenclature_2009.txt
-rw-r--r--    1 ftp      ftp         14582 Jan 16 15:56 README.md
drwxr-xr-x    2 ftp      ftp          3213 Jan 16 15:56 alignments
drwxr-xr-x    2 ftp      ftp          2062 Jan 16 15:56 allelelist
-rw-r--r--    1 ftp      ftp           113 Nov 30  2022 change_log.txt
drwxr-xr-x    2 ftp      ftp          3402 Jan 16 15:56 fasta
-rw-r--r--    1 ftp      ftp      268918026 Jan 16 15:56 hla.dat
-rw-r--r-- 

'226 Directory send OK.'

In [ ]:
file_to_download = 'hla_prot.fasta'

In [ ]:
local_file_path = '-rw-r--r--' + file_to_download

In [ ]:
ftp.retrbinary("RETR " + file_to_download ,open(file_to_download, 'wb').write)

'226 Transfer complete.'

## Converting File to pandas Dataframe

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.3 MB/s eta 0:00:00


In [ ]:
from Bio import SeqIO
import pandas as pd

def fasta_to_dataframe(fasta_file):
    records = SeqIO.parse(fasta_file, "fasta")
    data = []
    for record in records:
        sequence_id = record.id
        hla_info = " ".join(record.description.split()[1:-2])
        sequence = str(record.seq)
        sequence_length = len(sequence)
        data.append({"ID": sequence_id, "HLA": hla_info, "Sequence": sequence, "Length": sequence_length})
    df = pd.DataFrame(data)
    return df

fasta_file = "hla_prot.fasta"
df = fasta_to_dataframe(fasta_file)

print(df)

                 ID             HLA  \
0      HLA:HLA00001   A*01:01:01:01   
1      HLA:HLA02169  A*01:01:01:02N   
2      HLA:HLA14798   A*01:01:01:03   
3      HLA:HLA15760   A*01:01:01:04   
4      HLA:HLA16415   A*01:01:01:05   
...             ...             ...   
43265  HLA:HLA38075   TAP2*02:01:04   
43266  HLA:HLA38025      TAP2*02:02   
43267  HLA:HLA38029      TAP2*02:03   
43268  HLA:HLA38159      TAP2*02:04   
43269  HLA:HLA38463      TAP2*02:05   

                                                Sequence  Length  
0      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
1      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     200  
2      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
3      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
4      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
...                                                  ...     ...  
43265  MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...     7

### Testing Sequence Access

In [ ]:
R1_A2 = "A*32:01:01:01"

In [ ]:
def find_sequence(allele):
  HLA_allele = allele
  filtered_df = df[df['HLA'] == HLA_allele]

  filtered_rows = df[df['HLA'] == HLA_allele]
  filtered_df["Sequence"]

  return filtered_df

In [ ]:
def get_sequence(allele):
  s = find_sequence(allele)
  f = pd.Series(s.iloc[0])

  return f.loc['Sequence']

In [ ]:
recipient1_sequence = get_sequence(R1_A2)

In [ ]:
print(recipient1_sequence)

MAVMAPRTLLLLLLGALALTQTWAGSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDQETRNVKAHSQTDRESLRIALRYYNQSEAGSHTIQMMYGCDVGPDGRLLRGYQQDAYDGKDYIALNEDLRSWTAADMAAQITQRKWEAARVAEQLRAYLEGTCVEWLRRYLENGKETLQRTDAPKTHMTHHAVSDHEATLRCWALSFYPAEITLTWQRDGEDQTQDTELVETRPAGDGTFQKWASVVVPSGQEQRYTCHVQHEGLPKPLTLRWEPSSQPTIPIVGIIAGLVLFGAMFAGAVVAAVRWRRKSSDRKGGSYSQAASSDSAQGSDMSLTACKV


## Deleting Unnecessary HLA Sequence Rows

In [ ]:
df_filtered = df[df['HLA'].str.startswith('DMA')]

In [ ]:
print(df_filtered)

                 ID              HLA  \
42072  HLA:HLA00485  DMA*01:01:01:01   
42073  HLA:HLA06456  DMA*01:01:01:02   
42074  HLA:HLA06457  DMA*01:01:01:03   
42075  HLA:HLA06464  DMA*01:01:01:04   
42076  HLA:HLA34481  DMA*01:01:01:05   
42077  HLA:HLA34453  DMA*01:01:01:06   
42078  HLA:HLA34444  DMA*01:01:01:07   
42079  HLA:HLA34474  DMA*01:01:01:08   
42080  HLA:HLA34643  DMA*01:01:01:09   
42081  HLA:HLA34571  DMA*01:01:01:10   
42082  HLA:HLA34579  DMA*01:01:01:11   
42083  HLA:HLA34596  DMA*01:01:01:12   
42084  HLA:HLA34597  DMA*01:01:01:13   
42085  HLA:HLA34521  DMA*01:01:01:14   
42086  HLA:HLA34573  DMA*01:01:01:15   
42087  HLA:HLA34581  DMA*01:01:01:16   
42088  HLA:HLA34553  DMA*01:01:01:17   
42089  HLA:HLA34519  DMA*01:01:01:18   
42090  HLA:HLA34523  DMA*01:01:01:19   
42091  HLA:HLA34535  DMA*01:01:01:20   
42092  HLA:HLA34532  DMA*01:01:01:21   
42093  HLA:HLA34555  DMA*01:01:01:22   
42094  HLA:HLA34546  DMA*01:01:01:23   
42095  HLA:HLA34572  DMA*01:01:01:24   


In [ ]:
df = df[~df['HLA'].str.startswith('DMA')]

In [ ]:
print(df)

                 ID             HLA  \
0      HLA:HLA00001   A*01:01:01:01   
1      HLA:HLA02169  A*01:01:01:02N   
2      HLA:HLA14798   A*01:01:01:03   
3      HLA:HLA15760   A*01:01:01:04   
4      HLA:HLA16415   A*01:01:01:05   
...             ...             ...   
43265  HLA:HLA38075   TAP2*02:01:04   
43266  HLA:HLA38025      TAP2*02:02   
43267  HLA:HLA38029      TAP2*02:03   
43268  HLA:HLA38159      TAP2*02:04   
43269  HLA:HLA38463      TAP2*02:05   

                                                Sequence  Length  
0      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
1      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     200  
2      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
3      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
4      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
...                                                  ...     ...  
43265  MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...     7

In [ ]:
df = df[~df['HLA'].str.startswith('DMB')]

In [ ]:
print(df)

                 ID             HLA  \
0      HLA:HLA00001   A*01:01:01:01   
1      HLA:HLA02169  A*01:01:01:02N   
2      HLA:HLA14798   A*01:01:01:03   
3      HLA:HLA15760   A*01:01:01:04   
4      HLA:HLA16415   A*01:01:01:05   
...             ...             ...   
43265  HLA:HLA38075   TAP2*02:01:04   
43266  HLA:HLA38025      TAP2*02:02   
43267  HLA:HLA38029      TAP2*02:03   
43268  HLA:HLA38159      TAP2*02:04   
43269  HLA:HLA38463      TAP2*02:05   

                                                Sequence  Length  
0      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
1      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     200  
2      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
3      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
4      MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...     365  
...                                                  ...     ...  
43265  MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...     7

In [ ]:
df = df[~df['HLA'].str.startswith('DOA')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('DOB')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('DRA')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('E')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('F')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('G')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('HFE')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('MICA')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('MICB')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('TAP1')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
43265,HLA:HLA38075,TAP2*02:01:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43266,HLA:HLA38025,TAP2*02:02,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43267,HLA:HLA38029,TAP2*02:03,MRLPDLRPWTSLLLADAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704
43268,HLA:HLA38159,TAP2*02:04,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,704


In [ ]:
df = df[~df['HLA'].str.startswith('TAP2')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
1,HLA:HLA02169,A*01:01:01:02N,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,200
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
42067,HLA:HLA39147,DPB1*1574:01,NYLFQGRQECYAFNGTQRFLERYIYNREEFVRFDSDVGEFRAVTER...,224
42068,HLA:HLA39087,DPB1*1575:01,NYVYQLRQECYPFNGTQRFLERYIYNREEYARFDSDVGEFRAVTEL...,224
42069,HLA:HLA39088,DPB1*1576:01,NYVYQLRQECYAFNGTQRFLERYIYNWQEYARFDSDVGEFRAVTEL...,224
42070,HLA:HLA39106,DPB1*1577:01,NYVHQLRQECYAFNGTQRFLERYIYNREEFVRFDRDVGEFRAVTEL...,224


In [ ]:
df = df[~df['HLA'].str.endswith('N')]
df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
2,HLA:HLA14798,A*01:01:01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
3,HLA:HLA15760,A*01:01:01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
4,HLA:HLA16415,A*01:01:01:05,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
5,HLA:HLA16417,A*01:01:01:06,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
...,...,...,...,...
42067,HLA:HLA39147,DPB1*1574:01,NYLFQGRQECYAFNGTQRFLERYIYNREEFVRFDSDVGEFRAVTER...,224
42068,HLA:HLA39087,DPB1*1575:01,NYVYQLRQECYPFNGTQRFLERYIYNREEYARFDSDVGEFRAVTEL...,224
42069,HLA:HLA39088,DPB1*1576:01,NYVYQLRQECYAFNGTQRFLERYIYNWQEYARFDSDVGEFRAVTEL...,224
42070,HLA:HLA39106,DPB1*1577:01,NYVHQLRQECYAFNGTQRFLERYIYNREEFVRFDRDVGEFRAVTEL...,224


## Converting Into Field-2 Resolution

In [ ]:
unique_df = df[~df.duplicated(subset='Sequence', keep='first')]
unique_df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01:01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
119,HLA:HLA02540,A*01:01:04,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,181
120,HLA:HLA03131,A*01:01:05,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,273
138,HLA:HLA05685,A*01:01:23,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,312
174,HLA:HLA10775,A*01:01:59,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,337
...,...,...,...,...
42067,HLA:HLA39147,DPB1*1574:01,NYLFQGRQECYAFNGTQRFLERYIYNREEFVRFDSDVGEFRAVTER...,224
42068,HLA:HLA39087,DPB1*1575:01,NYVYQLRQECYPFNGTQRFLERYIYNREEYARFDSDVGEFRAVTEL...,224
42069,HLA:HLA39088,DPB1*1576:01,NYVYQLRQECYAFNGTQRFLERYIYNWQEYARFDSDVGEFRAVTEL...,224
42070,HLA:HLA39106,DPB1*1577:01,NYVHQLRQECYAFNGTQRFLERYIYNREEFVRFDRDVGEFRAVTEL...,224


In [ ]:
for index, allele in unique_df.iterrows():
    if len(allele['HLA'].split(':')) > 2:
        unique_df.at[index, 'HLA'] = ':'.join(allele['HLA'].split(':')[:2])

In [ ]:
unique_df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
119,HLA:HLA02540,A*01:01,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,181
120,HLA:HLA03131,A*01:01,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,273
138,HLA:HLA05685,A*01:01,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,312
174,HLA:HLA10775,A*01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,337
...,...,...,...,...
42067,HLA:HLA39147,DPB1*1574:01,NYLFQGRQECYAFNGTQRFLERYIYNREEFVRFDSDVGEFRAVTER...,224
42068,HLA:HLA39087,DPB1*1575:01,NYVYQLRQECYPFNGTQRFLERYIYNREEYARFDSDVGEFRAVTEL...,224
42069,HLA:HLA39088,DPB1*1576:01,NYVYQLRQECYAFNGTQRFLERYIYNWQEYARFDSDVGEFRAVTEL...,224
42070,HLA:HLA39106,DPB1*1577:01,NYVHQLRQECYAFNGTQRFLERYIYNREEFVRFDRDVGEFRAVTEL...,224


In [ ]:
is_nan = unique_df['HLA'].isna().any()

if is_nan:
    print("There are NaN values in the 'HLA' column.")
else:
    print("There are no NaN values in the 'HLA' column.")

There are no NaN values in the 'HLA' column.


In [ ]:
unique_df = unique_df[~unique_df.duplicated(subset='HLA', keep='first')]

In [ ]:
unique_df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
269,HLA:HLA00002,A*01:02,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFSTSVSRPGSGEPRF...,365
273,HLA:HLA00003,A*01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
279,HLA:HLA01031,A*01:06,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
280,HLA:HLA01208,A*01:07,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,181
...,...,...,...,...
42067,HLA:HLA39147,DPB1*1574:01,NYLFQGRQECYAFNGTQRFLERYIYNREEFVRFDSDVGEFRAVTER...,224
42068,HLA:HLA39087,DPB1*1575:01,NYVYQLRQECYPFNGTQRFLERYIYNREEYARFDSDVGEFRAVTEL...,224
42069,HLA:HLA39088,DPB1*1576:01,NYVYQLRQECYAFNGTQRFLERYIYNWQEYARFDSDVGEFRAVTEL...,224
42070,HLA:HLA39106,DPB1*1577:01,NYVHQLRQECYAFNGTQRFLERYIYNREEFVRFDRDVGEFRAVTEL...,224


In [ ]:
unique_df = unique_df[~unique_df['HLA'].str.endswith('Q')]
unique_df

,ID,HLA,Sequence,Length
0,HLA:HLA00001,A*01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
269,HLA:HLA00002,A*01:02,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFSTSVSRPGSGEPRF...,365
273,HLA:HLA00003,A*01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
279,HLA:HLA01031,A*01:06,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,365
280,HLA:HLA01208,A*01:07,SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEP...,181
...,...,...,...,...
42067,HLA:HLA39147,DPB1*1574:01,NYLFQGRQECYAFNGTQRFLERYIYNREEFVRFDSDVGEFRAVTER...,224
42068,HLA:HLA39087,DPB1*1575:01,NYVYQLRQECYPFNGTQRFLERYIYNREEYARFDSDVGEFRAVTEL...,224
42069,HLA:HLA39088,DPB1*1576:01,NYVYQLRQECYAFNGTQRFLERYIYNWQEYARFDSDVGEFRAVTEL...,224
42070,HLA:HLA39106,DPB1*1577:01,NYVHQLRQECYAFNGTQRFLERYIYNREEFVRFDRDVGEFRAVTEL...,224


## Pandas Dataframe to CSV File

In [ ]:
unique_df.to_csv('processed_sequences.csv')